In [ ]:
from pathlib import Path
import re
import pandas as pd
from matplotlib import colormaps
from matplotlib.colors import rgb2hex
from sklearn.preprocessing import MinMaxScaler
from matplotlib.animation import FuncAnimation
from matplotlib import pyplot as plt

def column_to_color(series, scale="viridis"):
    smin = series.min()
    smax = series.max()
    norm = (series - smin) / (smax - smin)
    cmap = colormaps[scale]
    return norm.apply(lambda x: rgb2hex(cmap(x)))

In [ ]:
data = Path("../wintab_lite/example/wintab_winit/run_output.log").read_text(encoding="UTF-16 LE")

In [ ]:
print(data[2000:3100])

In [ ]:
matches = re.findall(
    r"Packet \{.*?\},\n\}",
    data,
    flags=re.DOTALL
)
df = pd.Series(matches)
df

In [ ]:
time = df.str.extract(f"pkTime: (.*?),", expand=False).astype("u8").rename("time")
time


In [ ]:
xy = df.str.extract(r"xyz{ (.*?), (.*?), .*? }")
xy.columns = ["x", "y"]
xy = xy.astype("i4")
xy

In [ ]:
pressure = df.str.extract(r"pkNormalPressure: (.*?),", expand=False).astype("i4").rename("pressure")
pressure

In [ ]:
cdf = pd.concat([xy, time, pressure],axis="columns")
cdf=cdf[cdf["pressure"]>0]
cdf[["time","pressure"]] = MinMaxScaler().fit_transform(cdf[["time","pressure"]])
cdf.plot.scatter(x="x",y="y",c=column_to_color(cdf["time"]), s=cdf["pressure"]*20)

In [ ]:
cdf = pd.concat([xy, time, pressure],axis="columns")
cdf=cdf[cdf["pressure"]>0]
cdf[["pressure"]] = MinMaxScaler().fit_transform(cdf[["pressure"]])
cdf["time"] -= cdf["time"].min()

In [ ]:
cdf["time"].min(), cdf["time"].max()

In [ ]:
cdf.set_index("time")

In [ ]:
intervals = pd.interval_range(start=cdf["time"].min(), end=cdf["time"].max(), freq=10)
intervals

In [ ]:
frame_indexes = pd.Series(intervals).apply(lambda interval: cdf.index[cdf["time"].apply(lambda time: time in interval)])

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
scatter = ax.scatter([],[], c="red")


def init():
    ax.set_xlim(-2600, -1600)
    ax.set_ylim(700, 1500)
    return scatter,


def update(frame_indexes):
    print(frame_indexes)
    if len(frame_indexes) == 0:
        return
    dat=cdf.loc[frame_indexes]
    xy = dat.loc[:,["x","y"]].values
    size = dat.loc[:,"pressure"].values*30
    color = column_to_color(dat.loc[:,"pressure"]).values
    scatter.set_offsets(xy)
    scatter.set_sizes(size)
    scatter.set_color(color)
    return scatter,

animation = FuncAnimation(
    fig       = fig,
    func      = update,
    init_func = init,
    frames    = frame_indexes,
    interval  = 5
)
animation.save("animation.gif", writer="pillow")